In [1]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

#used to store model artifacts which SageMaker will extract to /opt/ml/model in the container, 
#in this example case we will not be making use of S3 to store the model artifacts
#s3_bucket = '<S3Bucket>'

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/mabedmutha/Library/Application Support/sagemaker/config.yaml


NoRegionError: You must specify a region.

In [4]:
from time import localtime, strftime

launch_time = strftime("%Y-%m-%d-%H-%M-%S", localtime())

base_name = 'sm-willo-recommendation-system'
model_name =  f'{base_name}-' + launch_time
container = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{base_name}:latest'
instance_type = 'ml.c5.large'

print('Model name: ' + model_name)
#print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container]
)

print("Model Arn: " + create_model_response['ModelArn'])

NameError: name 'account_id' is not defined

In [11]:
endpoint_config_name = f'{base_name}-config' + launch_time
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])
        
print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: sm-willo-recommendation-system-config2024-05-23-19-56-47
Endpoint config Arn: arn:aws:sagemaker:us-west-2:992382616215:endpoint-config/sm-willo-recommendation-system-config2024-05-23-19-56-47


In [12]:
%%time

import time

endpoint_name = f'{base_name}-endpoint' + launch_time
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)


Endpoint name: sm-willo-recommendation-system-endpoint2024-05-23-19-56-47
Endpoint Arn: arn:aws:sagemaker:us-west-2:992382616215:endpoint/sm-willo-recommendation-system-endpoint2024-05-23-19-56-47
Endpoint Status: Creating
Waiting for sm-willo-recommendation-system-endpoint2024-05-23-19-56-47 endpoint to be in service...
CPU times: user 31.1 ms, sys: 3.57 ms, total: 34.7 ms
Wall time: 2min 31s


In [14]:
import json
content_type = "application/json"
request_body = {"input": "user123"}

#Serialize data for endpoint
#data = json.loads(json.dumps(request_body))
payload = json.dumps(request_body)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload
)

#Parse results
result = json.loads(response['Body'].read().decode())
result

{'UserID': 'user123',
 'ResourceID': ['60354a15e9d133227dc9f5cc',
  '642c63868469673e35f3d3cf',
  '651ca09760198034773516c3'],
 'SessionID': 'session_user123'}

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)